In [10]:
import openai
import os
from openai import OpenAI



In [11]:
# Uncomment if you are in a Jupyter Notebook
import nest_asyncio
nest_asyncio.apply()

from llama_parse import LlamaParse  # pip install llama-parse
from llama_index.core import SimpleDirectoryReader  # pip install llama-index

parser = LlamaParse(
    api_key=key, # can also be set in your env as LLAMA_CLOUD_API_KEY
    result_type="markdown"  # "markdown" and "text" are available
)

file_extractor = {".pdf": parser}
reader = SimpleDirectoryReader("./files", file_extractor=file_extractor)
documents = reader.load_data()

Started parsing the file under job_id 6c980e94-93e1-49ab-bab9-b119df94b0e6
Started parsing the file under job_id 14dafe35-90be-4eb1-89df-54141170a723
Started parsing the file under job_id c221387f-da95-450d-9d3a-8e3c942c25b5
Started parsing the file under job_id da7ba09b-cbcf-48bd-956d-53f63a40693d
Started parsing the file under job_id 67797b13-7c80-4b64-bede-59d0dd3c9031
Started parsing the file under job_id 7254d450-601c-45a8-aabc-cde2f13350ad
Started parsing the file under job_id 7e41dbbf-80f8-4991-ae05-2f4aaa2daf5b
Started parsing the file under job_id 0cb72f59-bf77-40ab-89af-bb9eab9f115c
Started parsing the file under job_id 90fc2ded-1c19-496f-b7ff-ef01ce26a872


In [12]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.response.pprint_utils import pprint_response
from llama_index.llms.openai import OpenAI

from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine

In [13]:
import os

os.environ["OPENAI_API_KEY"] = key

In [15]:
from llama_index.core import Settings

Settings.llm = OpenAI(temperature=0.2, model="gpt-4o")

In [16]:
acciones = SimpleDirectoryReader(
    input_files=["./files/Acciones.pdf"]
).load_data()
bonos = SimpleDirectoryReader(
    input_files=["./files/Bonos.pdf"]
).load_data()
mate_fin_1 = SimpleDirectoryReader(
    input_files=["./files/Matemáticas Financieras Parte 1.pdf"]
).load_data()
mate_fin_2 = SimpleDirectoryReader(
    input_files=["./files/Matemáticas Financieras Parte 2.pdf"]
).load_data()
mate_fin_4 = SimpleDirectoryReader(
    input_files=["./files/Matemáticas Financieras Parte 4.pdf"]
).load_data()
mate_fin_5 = SimpleDirectoryReader(
    input_files=["./files/Matemáticas Financieras Parte 5.pdf"]
).load_data()
valuación = SimpleDirectoryReader(
    input_files=["./files/Valuación de Empresas DCF-MúltiplosFF.pdf"]
).load_data()

In [17]:
acciones_index = VectorStoreIndex.from_documents(acciones)
bonos_index = VectorStoreIndex.from_documents(bonos)
mate_fin_1_index = VectorStoreIndex.from_documents(mate_fin_1)
mate_fin_2_index = VectorStoreIndex.from_documents(mate_fin_2)
mate_fin_4_index = VectorStoreIndex.from_documents(mate_fin_4)
mate_fin_5_index = VectorStoreIndex.from_documents(mate_fin_5)
valuación_index = VectorStoreIndex.from_documents(valuación)

In [18]:
acciones_engine = acciones_index.as_query_engine(similarity_top_k=3)
bonos_engine = bonos_index.as_query_engine(similarity_top_k=3)
mate_fin_1_engine = mate_fin_1_index.as_query_engine(similarity_top_k=3)
mate_fin_2_engine = mate_fin_2_index.as_query_engine(similarity_top_k=3)
mate_fin_4_engine = mate_fin_4_index.as_query_engine(similarity_top_k=3)
mate_fin_5_engine = mate_fin_5_index.as_query_engine(similarity_top_k=3)
valuación_engine = valuación_index.as_query_engine(similarity_top_k=3)


In [19]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=acciones_engine,
        metadata=ToolMetadata(
            name="acciones",
            description=(
                "Provides information about Actions"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=bonos_engine,
        metadata=ToolMetadata(
            name="bonos",
            description=(
                "Provides information about Bonds"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=mate_fin_1_engine,
        metadata=ToolMetadata(
            name="mate_fin_1",
            description=(
                "Provides information about Financial Mathematics Part 1"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=mate_fin_2_engine,
        metadata=ToolMetadata(
            name="mate_fin_2",
            description=(
                "Provides information about Financial Mathematics Part 2"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=mate_fin_4_engine,
        metadata=ToolMetadata(
            name="mate_fin_4",
            description=(
                "Provides information about Financial Mathematics Part 4"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=mate_fin_5_engine,
        metadata=ToolMetadata(
            name="mate_fin_5",
            description=(
                "Provides information about Financial Mathematics Part 5"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=valuación_engine,
        metadata=ToolMetadata(
            name="valuación",
            description=(
                "Provides information about Company Valuation DCF-MúltiplosFF"
            ),
        ),
    ),
]

In [20]:
s_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools
)

In [21]:
response = s_engine.query(
    """La Corporación ARCH implementó a fines de 2021 un nuevo proyecto que implicó una inversión en ese momento de $150 millones. Para evaluar esta nueva inversión, el administrador financiero obtuvo las siguientes proyecciones para el cierre de cada año:

($ millones)

2022

2023

Ventas

180

214

Costos de ventas y administración (no incluye depreciación)

77

93

Depreciación

14

21

Gastos de capital (CAPEX)

25

33

Inversión en capital de trabajo neto

32

28

Además, el proyecto implica el uso de una bodega para almacenamiento del equipamiento de producción. De no realizarse el proyecto, esta bodega podría rentarse y generar un flujo de efectivo después de impuestos de $8 millones por año (comenzando en 2022).

Después de 2023, los flujos de efectivo libres crecerán al 4% efectivo anual a perpetuidad.

El costo promedio ponderado del capital (WACC) será del 10% efectivo anual durante los primeros dos años, y posteriormente será del 12% efectivo anual.

La tasa de impuestos corporativa es del 30%.

Obtenga el Índice de Rentabilidad a finales de 2021 para este proyecto, considerando el VPN en el numerador, es decir VPN/|I0|, donde I0: inversión inicial""")

Generated 6 sub questions.
[mate_fin_1] Q: ¿Cuál es la fórmula para calcular el Valor Presente Neto (VPN) de un proyecto de inversión?
[mate_fin_1] Q: ¿Cómo se calcula el Índice de Rentabilidad (IR) de un proyecto de inversión?
[mate_fin_2] Q: ¿Qué es el WACC y cómo se utiliza en la evaluación de proyectos de inversión?
[mate_fin_4] Q: ¿Cómo se calcula el flujo de efectivo libre (FCF) de un proyecto de inversión?
[mate_fin_5] Q: ¿Cómo se considera el crecimiento perpetuo de los flujos de efectivo libres en la evaluación de proyectos?
[mate_fin_4] Q: ¿Cómo se incorpora la tasa de impuestos corporativa en el cálculo del flujo de efectivo libre?
[mate_fin_4] A: Para incorporar la tasa de impuestos corporativa en el cálculo del flujo de efectivo libre, se debe ajustar el ingreso operativo neto restando los impuestos corporativos. Esto se hace multiplicando el ingreso operativo neto por (1 - tasa de impuestos corporativa). Luego, se suman las depreciaciones y amortizaciones, se restan los c

In [22]:
print(response)

Para calcular el Índice de Rentabilidad (IR) a finales de 2021 para el proyecto de la Corporación ARCH, primero debemos determinar el Valor Presente Neto (VPN) de los flujos de efectivo libres proyectados. Aquí están los pasos detallados:

1. **Calcular el Flujo de Efectivo Libre (FCF) para 2022 y 2023:**

\[ FCF = EBIT (1 - Tasa\ de\ impuestos) + Depreciación - Gastos\ de\ capital - Cambio\ en\ el\ capital\ de\ trabajo \]

Donde:
- EBIT = Ventas - Costos de ventas y administración - Depreciación
- Tasa de impuestos = 30%
- Depreciación, Gastos de capital (CAPEX), y Cambio en el capital de trabajo son dados.

Para 2022:
- Ventas = 180 millones
- Costos de ventas y administración = 77 millones
- Depreciación = 14 millones
- Gastos de capital (CAPEX) = 25 millones
- Cambio en el capital de trabajo = 32 millones

\[ EBIT_{2022} = 180 - 77 - 14 = 89 \]
\[ FCF_{2022} = 89 \times (1 - 0.30) + 14 - 25 - 32 = 62.3 + 14 - 25 - 32 = 19.3 \]

Para 2023:
- Ventas = 214 millones
- Costos de ventas 